In [14]:
import geopandas as gpd
import classes.entropycalculator as ec
import gc
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy

categorisation = pd.read_excel("data/categorisation.xlsx")

gm_list = ['Eindhoven', 'Amsterdam', 'Rotterdam', 'Utrecht', 'Groningen', 'Maastricht', 'Leeuwarden', 'Arnhem', 'Zwolle', "'s-Gravenhage"]
scales = ['wijken', 'buurten']
filters = [0, 1, 2]

In [15]:
def get_categorisation_filtered(categorisation, L0_filter, L1_filter):
    # Filter the categorisation dataframe based on the L0 and L1 categories
    categorisation = categorisation[~categorisation["L0 category"].isin(L0_filter)]
    if L1_filter:
        for key, value in L1_filter.items():
            categorisation = categorisation[
                ~(
                    (categorisation["L0 category"] == key)
                    & (categorisation["L1 category"].isin(value))
                )
            ]
    L0_cats = categorisation["L0 category"].unique()
    L1_cats = categorisation["L1 category"].unique()
    return L0_cats, L1_cats

In [16]:
def append_amenity_counts(gemeente):
    
    amenities_gm = gpd.read_parquet(f"data/gm_amenities/amenities_{gemeente}.parquet")
    
    for filteri in filters:
        
        L0_filter, L1_filter = ec.getfilter(filteri)
        L0_cats, L1_cats = get_categorisation_filtered(categorisation, L0_filter, L1_filter)
        
        for scale in scales:
            
            gdf = gpd.read_parquet(f"results/filter{filteri}/{scale}/{gemeente}_{scale}_{filteri}.parquet")
            
            for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
                amenities = amenities_gm[amenities_gm.within(row["geometry"])]
                
                if amenities.empty:
                    for cat in L0_cats:
                        gdf.at[idx, f"L0_c_{cat}"] = 0
                    for cat in L1_cats:
                        gdf.at[idx, f"L1_c_{cat}"] = 0
                    continue
                
                L0_counts, L1_counts = amenities.value_counts("L0_category"), amenities.value_counts("L1_category")
                
                for cat in L0_cats:
                    gdf.at[idx, f"L0_c_{cat}"] = L0_counts.get(cat, 0)
                    
                for cat in L1_cats:
                    gdf.at[idx, f"L1_c_{cat}"] = L1_counts.get(cat, 0)
                    
            gdf.to_parquet(f"results/filter{filteri}/{scale}/{gemeente}_{scale}_{filteri}.parquet")
            
            del gdf
            del amenities
            del L0_counts
            del L1_counts
            gc.collect()
            

In [19]:
for gm in gm_list:
    print(f"Processing {gm}")
    append_amenity_counts(gm)

Processing Eindhoven


100%|██████████| 112/112 [00:00<00:00, 233.53it/s]


Processing Amsterdam


100%|██████████| 489/489 [00:02<00:00, 177.72it/s]


Processing Rotterdam


...

100%|██████████| 83/83 [00:00<00:00, 207.58it/s]


Processing Zwolle


100%|██████████| 77/77 [00:00<00:00, 184.07it/s]


Processing 's-Gravenhage


100%|██████████| 111/111 [00:02<00:00, 42.68it/s]
